---

## A - OFDM Python Lab (AUTO-GRADER)

This notebook is intended to be an OFDM modulator with BPSK symbols, which shall be used in order to automatically modulate UTF-8 (text) strings and transmit them over a virtual AWGN channel.

* Generate num_homeworks random strings that shall be transmitted 
* BPSK-modulate the generated UTF-8 symbols 
* OFDM-arrange the BPSK symbols 
* Transmit them over a multipath and AWGN virtual channel and save the result to an .npy file


---


In [1]:
import numpy as np
import helper_functions as hf
import matplotlib.pyplot as plt

<code>ofdm_facts</code> contains 50 messages that are going to be transmitted for each homework (generated by GPT 3.5) <br>
<code>num_homeworks</code> contains the number of homeworks to be generated

In [2]:
num_homeworks = 50
ofdm_facts = [
        "OFDM stands for Orthogonal Frequency Division Multiplexing.",
        "OFDM is a modulation technique used in wireless communication systems.",
        "It is widely employed in Wi-Fi, 4G LTE, 5G, and digital television broadcasting.",
        "OFDM divides the available spectrum into multiple orthogonal subcarriers.",
        "Orthogonality between subcarriers reduces interference and enhances spectral efficiency.",
        "OFDM is known for its resilience against frequency-selective fading and multipath interference.",
        "It is a key technology in achieving high data rates in broadband communication.",
        "OFDM enables simultaneous transmission of data on multiple subcarriers.",
        "FFT (Fast Fourier Transform) is used in OFDM for modulation and demodulation.",
        "Guard intervals are inserted between OFDM symbols to mitigate inter-symbol interference.",
        "OFDM is used in both downlink and uplink communication in cellular networks.",
        "MIMO (Multiple Input Multiple Output) is often combined with OFDM for enhanced performance.",
        "OFDM is used in digital audio broadcasting standards like DAB and DRM.",
        "One of the first widely adopted OFDM standards was DVB-T for digital television.",
        "OFDM is part of the IEEE 802.11a/g/n/ac Wi-Fi standards.",
        "LTE (Long-Term Evolution) uses OFDM for downlink and SC-FDMA for uplink.",
        "5G NR (New Radio) uses OFDM as its core modulation scheme.",
        "OFDM allows for dynamic allocation of subcarriers based on channel conditions.",
        "Cyclic Prefix in OFDM helps mitigate the effects of multipath fading.",
        "OFDM systems are often susceptible to peak-to-average power ratio (PAPR) issues.",
        "Adaptive modulation and coding can be employed in OFDM systems for efficient spectrum usage.",
        "OFDM is a key technology in COFDM (Coded Orthogonal Frequency Division Multiplexing).",
        "OFDM is used in digital subscriber line (DSL) technologies like VDSL2.",
        "WiMAX, a wireless broadband technology, utilizes OFDM for efficient data transmission.",
        "OFDM is known for its robustness in non-line-of-sight (NLOS) environments.",
        "OFDM is a key technology for broadband power line communication (BPLC).",
        "ITU-T G.hn standard for home networking uses OFDM for high-speed data transmission.",
        "OFDM is essential in achieving efficient spectrum utilization in cognitive radio networks.",
        "OFDM is employed in radar systems for its ability to provide high range resolution.",
        "DAB+ (Digital Audio Broadcasting Plus) uses OFDM for audio broadcasting.",
        "OFDM is utilized in underwater acoustic communication systems.",
        "OFDM enables efficient transmission over frequency-selective fading channels.",
        "OFDM is a popular choice for digital video broadcasting (DVB).",
        "OFDM systems often employ error correction coding to enhance reliability.",
        "OFDM is used in power distribution systems for smart grid communication.",
        "OFDM is a key technology for achieving high spectral efficiency in satellite communication.",
        "OFDM can be susceptible to narrowband interference, requiring advanced mitigation techniques.",
        "OFDM can support different QAM (Quadrature Amplitude Modulation) orders for varying data rates.",
        "OFDM is used in vehicular communication systems for high-speed data exchange.",
        "OFDM is employed in indoor optical wireless communication systems.",
        "OFDM is a critical technology for achieving low-latency communication in industrial IoT.",
        "OFDM is utilized in passive optical networks (PONs) for efficient fiber-optic communication.",
        "OFDM is employed in digital radio mondiale (DRM) for long-distance broadcasting.",
        "OFDM is a key technology for achieving high data rates in airborne communication.",
        "OFDM can be combined with spread spectrum techniques for enhanced performance.",
        "OFDM plays a crucial role in achieving high-speed communication in space missions.",
        "OFDM is a fundamental technology for achieving reliable communication in harsh environments.",
        "OFDM is used in audio watermarking for copyright protection.",
        "OFDM is employed in digital terrestrial television broadcasting standards like ATSC 3.0."
    ]
prefix = ("Congratulations on completing the OFDM lab! OFDM is an amazing technology with numerous applications, did you know that ")
ofdm_facts_prefix = [prefix + fact for fact in ofdm_facts]

OFDM parameters can be set below

In [3]:
fs = 20e6 # Sampling rate 
N = 64 # No. of sub-carriers 
n_data = 52 # No. of data sub-carriers
cp_len = 16 # No. of cyclic prefix samples
n_ofdm = 1000 # No. of OFDM symbols
nsym = n_ofdm * n_data # No. of data symbols

Modulate the facts using BPSK symbols (see <code>helper_functions.py</code> for the code) - text is represented in UTF-8

In [4]:
padded_bpsk_symbols = np.array([])

for fact in ofdm_facts_prefix:
    bpsk_symbols = hf.bpsk_gen_text(fact)
    padding = nsym - bpsk_symbols.size
    padded_fact = np.pad(bpsk_symbols, (0, padding), constant_values=-1)
    padded_bpsk_symbols = np.append(padded_bpsk_symbols,padded_fact)

padded_bpsk_symbols = padded_bpsk_symbols.reshape(-1, nsym)

Preamble generation

In [7]:
#L-LTF sequence
LTFseq = np.array([0,0,0,0,0,0,1,1,-1,\
                  -1,1,1,-1,1,-1,1,1,1,\
                  1,1,1,-1,-1,1,1,-1,1,\
                  -1,1,1,1,1,0,1,-1,-1,\
                  1,1,-1,1,-1,1,-1,-1,\
                  -1,-1,-1,1,1,-1,-1,1,\
                  -1,1,-1,1,1,1,1,0,0,0,\
                  0,0])

In [8]:
#preamble_symbols = np.fft.ifft(np.fft.fftshift(preamble_bits),N)

LTFsymb = np.fft.ifft(np.fft.fftshift(LTFseq),N)

# Extract 32 sample CP 
LTFcp = LTFsymb[32:64]

# Concatenate to form L-LTF
LLTF = np.concatenate((LTFcp, LTFsymb, LTFsymb))

# preamble = concat(preamble symbols + repeat 8 times + cyclic prefix=first 16 samples)
# 

In [9]:
# Indices for data sub-carriers (Guard/Unused Subcarriers)
ind_1 = np.arange(start=6, stop=32)
ind_2 = np.arange(start=33, stop=59)
index = np.concatenate((ind_1, ind_2), axis=0)

def ofdm_modulate(data):
    # Initialisation of array to hold OFDM symbols 
    ofdm_data = np.zeros(n_ofdm*N,np.complex64)
    j = 0 
    k = 0
    l = 0


    for i in range(n_ofdm):
        # Initialise array to hold data and null sub-carriers
        # (all null to begin with)
        sc_array = np.zeros(N,np.complex64)
    
        # Map data symbols to correct sub-carrier positions
        #print("i: " + str(i) + " j: " + str(j+n_data))
        #print(j+n_data)

        # FIR filter in the frequency domain

        sc_array[index] = data[j:j+n_data] #multiply each index with FIR filter here
    
        # Perform IFFT modulation
        ofdm_data[k:k+N] = np.fft.ifft(np.fft.fftshift(sc_array),N) 
    
        # Increment
        j = j + n_data
        k = k + N
        
    return ofdm_data

In [10]:
# Define function to add CP 
def add_cp(ofdm_symb,N,cp_len):
    
    #Extract CP
    cp = ofdm_symb[N-cp_len:N:1]
    
    # Concatenate CP and symbol 
    ofdm_symb_cp = np.concatenate((cp,ofdm_symb))
    
    return ofdm_symb_cp

cp_len = N // 4 # CP length is 1/4 of symbol period

def add_cp_ofdm_symbol(ofdm_data):
    # Add CP to each of the ofdm symbols 
    ofdm_data_cp = np.zeros(n_ofdm*(N+cp_len),np.complex64)
    j = 0
    k = 0 

    for i in range(n_ofdm):    
        ofdm_data_cp[k:(k+N+cp_len)] = add_cp(ofdm_data[j:j+N],N,cp_len)
        j = j + N  
        k = k + N + cp_len
    
    return ofdm_data_cp


Implementation of multipath + AWGN channel simulator

In [11]:
def multipath_fir_filter(signal):
    # Filter coefficients
    ntap = 4
    h = np.random.randn(ntap) + np.random.randn(ntap)*1j

    # Appy channel filter 
    txSig_filt = np.convolve(signal, h)
    return txSig_filt

def awgn_filter(signal, SNR=25):
    rxSig = hf.awgn(signal,SNR)
    return rxSig

This module saves the OFDM symbols to an npy file, which can be opened by students

In [13]:
#ofdm_modulated_strings = [ofdm_modulate(bpsk_strings) for bpsk_strings in padded_bpsk_symbols]

# Apply channel effect in time domain

save_idx = 0
for bpsk_strings in padded_bpsk_symbols:
    title = 'homework'+str(save_idx)+'.npy'
    array = add_cp_ofdm_symbol(ofdm_modulate(bpsk_strings))
    tx_signal = np.concatenate((LLTF, array))

    multipath_signal = multipath_fir_filter(tx_signal)
    awgn_rx_signal = awgn_filter(multipath_signal)

    np.save(title, tx_signal)
    save_idx += 1